In [4]:

import torch

In [5]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
from torch.autograd import Variable 
from torchvision import transforms 
from build_vocab import build_vocab, Vocabulary
from model import EncoderCNN, DecoderRNN
from model import ResNet, ResidualBlock
from PIL import Image
from attn_model import ResidualBlock, AttnEncoder, AttnDecoderRnn

embed_size = 128
hidden_size = 256 
num_layers = 1 
feature_size = 128

decoder_path = './models/visinsight_attn/decoder-10-9950.pkl'
encoder_path = './models/visinsight_attn/encoder-10-9950.pkl'


def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    image = image.resize([128, 128], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image

transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize((0.033, 0.032, 0.033), 
                         (0.027, 0.027, 0.027))])



vocab_path = './data/train1/vocab.pkl'
with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)
len_vocab = vocab.idx

# Build Models
encoder = AttnEncoder(ResidualBlock, [3, 3, 3])
encoder.eval()  # evaluation mode (BN uses moving mean/variance)
decoder = AttnDecoderRnn(feature_size, hidden_size, 
                     len(vocab), num_layers)


# Load the trained model parameters
encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))

# If use gpu
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

    

In [11]:
test_dir = 'data/validation2/png/'
# cap_dir = 'data/2object_test/caption/'
test_list = os.listdir(test_dir)
cnt = 0 

test_result = {}
for fname in test_list: 
    #cnt+=1 
    if cnt > 2:
        break;
    test_path = test_dir + fname
    test_image = load_image(test_path, transform)
    image_tensor = to_var(test_image)
    
    # Generate caption from image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    ids_arr = []
    for element in sampled_ids: 
        temp = element.cpu().data.numpy()
        ids_arr.append(int(temp))
#     print(fname)

    # Decode word_ids to words
    sampled_caption = []
    for word_id in ids_arr:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    in_caption = sampled_caption[1:-1]
#     print(in_caption)
    
#     print('------------------------------------')
    test_result[fname] = in_caption
    


In [12]:
import json
with open('test_result_attn.json', 'w') as f:
    json.dump(test_result, f)

In [ ]:
        
print('shape match: %.4f, location match: %.4f,location match_rect: %.4f, radius_match: %.4f, rect_x_match: %.4f, rect_y_match: %.4f color_match: %.4f'
     %(shape_match/(len(test_list)*2), location_match/(len(test_list)*2), location_match_rect/(len(test_list)*2),
      radius_match/len(test_list), rect_x_match/len(test_list), rect_y_match/len(test_list), color_match/(len(test_list)*2)))


In [ ]:
x_ = {x:rect_x_err.count(x) for x in rect_x_err}
y_ = {x:rect_y_err.count(x) for x in rect_y_err}
r_ = {x:radius_err.count(x) for x in radius_err}
loc_ ={x:location_err.count(x) for x in location_err}
loc_rect_ = {x:location_err_rect.count(x) for x in location_err_rect}

In [ ]:
x_

In [ ]:
test_dir = 'data/2object_test/bitmap/'
cap_dir = 'data/2object_test/caption/'
test_list = os.listdir(test_dir)
cnt = 0 

for fname in test_list: 
    test_path = test_dir + fname
    test_image = load_image(test_path, transform)
    image_tensor = to_var(test_image)
    
    # Generate caption from image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    ids_arr = []
    for element in sampled_ids: 
        temp = element.cpu().data.numpy()
        ids_arr.append(int(temp))


    # Decode word_ids to words
    sampled_caption = []
    for word_id in ids_arr:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    sentence = ' '.join(sampled_caption)
    print(sentence)
    predict = sentence.split()[1]
    
    cap_target = cap_dir+fname
    cap_path = cap_target.split('.')[0]+'.svg'
    with open(cap_path, 'r') as f:
        target = f.readline()
        print('target: '+ target)
    if predict == target:
        print('corret')
    #    cnt+=1 
        
print(cnt)
    
    
    

In [ ]:
feature.unsqueeze(1)

In [ ]:
cap_path = img_list[0].split('.')[0]+'.svg'

In [ ]:
cap_path